In [1]:
import pandas as pd

In [2]:
file = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_batting_df.csv'
file2 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_piching_df.csv'
file3 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_game_df.csv'

In [3]:
batting_df = pd.read_csv(file)
pitching_df = pd.read_csv(file2)
game_df = pd.read_csv(file3)

In [4]:
batting_df.drop('Unnamed: 0', axis=1, inplace=True)
pitching_df.drop('Unnamed: 0', axis=1, inplace=True)
game_df.drop('Unnamed: 0', axis=1, inplace=True)

## Batting Feature engineering ideas
- All of these stats will be on a PER PLATE APPEARANCE then divide by 162 to get PER GAME STATS
    - Team stats will be summed up and averaged - Will ultimately be compared to a teams 7 day trends history
        - 'totalBases' / total 'plateAppearances' - DONE
        - 'leftonbase'/ total 'plateAppearances' - DONE
        - 'runs' / total 'plateAppearances' - DONE
        - 'strikeOuts' / total 'atBats' - DONE
        - 'baseOnBalls' + 'intentionalWalks' / total 'plateAppearances' - DONE
            - 'hits' / total 'atBats' --- Leverage for game_df. get team stats per game...
            - 'obp' / # of batters on team --- Same as above
        - 'rbi' / total 'atBats' - DONE

In [5]:
batting_df.head()

,year,playerName,teamAbbrev,plateAppearances,totalBases,leftOnBase,runs,strikeOuts,baseOnBalls,intentionalWalks,hits,avg,atBats,obp,slg,ops,groundIntoDoublePlay,rbi
0,2009,Aaron Hill,TOR,734,340,286,103,98,42,1,195,0.286,682,0.330,0.499,0.829,17,108
1,2009,Jimmy Rollins,PHI,725,284,224,100,70,44,1,168,0.250,672,0.296,0.423,0.719,7,77
2,2009,Orlando Cabrera,MIN,708,255,244,83,71,36,1,186,0.284,656,0.316,0.389,0.705,22,77
3,2009,Nick Markakis,BAL,711,291,286,94,98,56,0,188,0.293,642,0.347,0.453,0.801,12,101
4,2009,Ichiro Suzuki,SEA,678,297,176,88,71,32,15,225,0.352,639,0.386,0.465,0.851,1,46


In [6]:
# MIA & FLO are different. Standardizing to MIA
batting_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [7]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(batting_df.teamAbbrev.unique())

31

### Create New Features for TEAM stats

In [8]:
# THIS IS THE FRAMEWORK TO GENERATE TEAM STATISTICS...
# Creating a dataframe containing team annual Batting Averages
a = batting_df.groupby(['year', 'teamAbbrev'])['hits'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BA = a.join(b, on=['year', 'teamAbbrev'])
BA['team_BA'] = BA['hits']/BA['plateAppearances']
BA = BA.reset_index()

In [9]:
# Total bases per plate appearance
# Commented out. Taking a different approach
# batting_df['Total_bases_per_plateappearance'] = (batting_df['totalBases'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['totalBases'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
TB = a.join(b, on=['year', 'teamAbbrev'])
TB['total_bases_avg'] = TB['totalBases']/TB['plateAppearances']
TB = TB.reset_index()

In [10]:
# LOB per plate appearnace
# Commented out. Taking a different approach
# batting_df['LOB_per_plateappearance'] = (batting_df['leftOnBase'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['leftOnBase'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
LOB = a.join(b, on=['year', 'teamAbbrev'])
LOB['LOB_avg'] = LOB['leftOnBase']/LOB['plateAppearances']
LOB = LOB.reset_index()

In [11]:
# Runs per plate appearance
# Commented out. Taking a different approach
#batting_df['runs_per_plateappearance'] = (batting_df['runs'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['runs'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RUNS = a.join(b, on=['year', 'teamAbbrev'])
RUNS['runs_avg'] = RUNS['runs']/RUNS['plateAppearances']
RUNS = RUNS.reset_index()

In [12]:
# K's per AtBat
# Commented out. Taking a different approach
# batting_df['K_per_plateappearance'] = (batting_df['strikeOuts'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['strikeOuts'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['atBats'].sum()
a = a.to_frame()
b = b.to_frame()
K = a.join(b, on=['year', 'teamAbbrev'])
K['K_avg'] = K['strikeOuts']/K['atBats']
K = K.reset_index()

In [13]:
# BB per plate appearance
# Commented out. Taking a different approach
# batting_df['BB+IBB_per_plateappearance'] = ((batting_df['baseOnBalls'] + 'took out IBB for simplicity' / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['baseOnBalls'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BB = a.join(b, on=['year', 'teamAbbrev'])
BB['BB_avg'] = BB['baseOnBalls']/BB['plateAppearances']
BB = BB.reset_index()

In [14]:
# RBI per plate appearance
# Commented out. Taking a different approach
# batting_df['RBI_per_plateappearance'] = (batting_df['rbi'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['rbi'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RBI = a.join(b, on=['year', 'teamAbbrev'])
RBI['RBI_avg'] = RBI['rbi']/RBI['plateAppearances']
RBI = RBI.reset_index()

## Merge all the TEAM stats into one dataframe

In [15]:
# team_stats_lst = [BA, TB, LOB, RUNS, K, BB, RBI]

In [16]:
team_stats = BA.merge(TB, on='year')

In [17]:
team_stats = team_stats.merge(LOB, on='year')

In [18]:
team_stats = team_stats.merge(RUNS, on='year')

In [19]:
team_stats = team_stats.merge(K, on='year')

KeyboardInterrupt: 

In [ ]:
team_stats = team_stats.merge(BB, on='year')

In [ ]:
team_stats = team_stats.merge(RBI, on='year')

In [ ]:
team_stats

## Pitching Feature engineering ideas
- Pitching stats will focus on starting pitchers Per9 innings stat (except for certain features)
    - 'winningPercentage'
    - 'runsScoredPer9'
        - 'ops' - may leave out for now...
    - 'hitsPer9'
    - 'era'
    - 'whip'

In [ ]:
pitching_df.head()

In [ ]:
pitching_df = pitching_df.reindex(columns=['year', 'playerName', 'teamAbbrev', 'winningPercentage',
       'runsScoredPer9', 'hitsPer9', 'strikeoutsPer9', 'baseOnBallsPer9',
       'homeRunsPer9', 'era', 'whip', 'ops', 'gidp', 'avg'])
# Columns to be left out from the reindexing --- 'winPercentage', 'strikeoutsPer9Inn', 'walksPer9Inn', 'hitsPer9Inn'

In [ ]:
# MIA & FLO are different. Standardizing to MIA
pitching_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [ ]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(pitching_df.teamAbbrev.unique())

## Game DF ideas
- Turn VisitorRunsScored & HomeRunsScore into a column that indicates if the home team won - DONE.
- Combine VisitorHBP & VisitorBB & VisitorIBB into a single column containing all the stats - DONE.
- Same with HomeHBP & HomeBB & HomeIBB - DONE.
<br/><br/>
- Combine VisitorH / VisitorAB = team_BA - DONE.
- Same with HomeH / HomeAB - DONE. 
<br/><br/>
- Eventually add on all the other engineered features into this dataframe...
<br/><br/>
- Since the game_df has a date column, I'd like to leverage that to look at 7-day-trends, 14-day-trends, etc

In [ ]:
game_df

In [ ]:
# Creating a Target for the model to eventually train on
game_df['Home_team_won?'] = game_df['HomeRunsScore'] > game_df['VisitorRunsScored']

In [ ]:
# game_df['Visitor_HBP+BB+IBB'] = game_df['VisitorHBP'] + game_df['VisitorBB'] + game_df['VisitorIBB']
# game_df['Home_HBP+BB+IBB'] = game_df['HomeHBP'] + game_df['HomeBB'] + game_df['HomeIBB']

# Commenting out for now until I figure out if I need to drop these or not...
# game_df.drop(['VisitorHBP', 'VisitorBB', 'VisitorIBB', 'HomeHBP', 'HomeBB', 'HomeIBB'], axis=1, inplace=True)

In [ ]:
# Adding a Home & Away BA per game to compare batting trends over time
game_df['Visitor_BA'] = round((game_df['VisitorH'] / game_df['VisitorAB']), 3)
game_df['Home_BA'] = round((game_df['HomeH'] / game_df['HomeAB']), 3)

In [ ]:
# Creating a new column in order to add Date/Time to my dataframe
game_df['New_Date'] = pd.to_datetime(game_df['Date'].astype(str), format='%Y%m%d')

In [ ]:
# Rearranging columns so new date/time is at the front of the DF
game_df = game_df.reindex(columns=['New_Date','Date', 'VisitingTeam','VisitorStartingPitcherName',
                                 'HomeTeam','HomeStartingPitcherName', 'VisitorRunsScored',
       'HomeRunsScore', 'VisitorAB', 'VisitorH','Visitor_BA', 'VisitorHBP', 'VisitorBB',
       'VisitorIBB', 'VisitorLOB', 'HomeAB', 'HomeH', 'Home_BA', 'HomeHBP', 'HomeBB', 'HomeIBB', 'HomeLOB', 'Home_team_won?'])
game_df.drop('Date', axis='columns', inplace=True)

In [ ]:
# Had an extra team due to the Marlins having two different labels. Updating their team name to MIA
game_df.replace(to_replace='FLO', value='MIA', inplace=True)
# Updating game_df so team abbreviations match. Will be leveraged when aggregating teams stats
game_df.replace(to_replace={"NYA":"NYY", "SDN":"SD", "CHN":"CHC", "SLN":"STL", "SFN":"SF", "LAN":"LAD", "TBA":"TB", "KCA":"KC", "CHA":"CWS", "ANA":"LAA", "NYN":"NYM"}, inplace=True)

In [ ]:
# game_df has 30 teams. Perfect!
len(game_df.VisitingTeam.unique())